# End to end experiment: Github Issue Summarization

Currently, this notebook must be run from the Kubeflow JupyterHub installation, as described in the codelab.

In this notebook, we will show how to:

* Interactively define a KubeFlow Pipeline using the Pipelines Python SDK
* Submit and run the pipeline
* Add a step in the pipeline

This example pipeline trains a [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor/) model on Github issue data, learning to predict issue titles from issue bodies. It then exports the trained model and deploys the exported model to [Tensorflow Serving](https://github.com/tensorflow/serving). 
The final step in the pipeline launches a web app which interacts with the TF-Serving instance in order to get model predictions.

## Enviroinment Setup

Before any experiment can be conducted. We need to setup and initialize an environment: ensure all Python modules has been setup and configured, as well as python modules

Setting up python modules

In [ ]:
print("Initializing...")
import sys
if sys.version_info.major != 3:
    raise ValueError('We only support Python 3; recommended Python 3.6')
# sys.path.insert(0, 'src')

!pip3 install --upgrade 'pip' > /dev/null
!pip3 install --upgrade 'https://storage.googleapis.com/ml-pipeline/release/0.1.16/kfp.tar.gz' > /dev/null
!pip3 install --upgrade './extensions' > /dev/null

# %load_ext autoreload
# %autoreload 2

%load_ext extensions

import boto3, re

import kfp
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.notebook

from ipython_secrets import get_secret
from kfp.compiler import Compiler

import extensions
import extensions.kaniko as kaniko
import extensions.pv as pv
import extensions.kubernetes as k8s
import extensions.kaniko.aws as aws

from os import environ

print("Done!")

Initialize global namespace variables. It is a good practice to place all global namespace variables in one cell. So, the notebook could be configured all-at-once. 

To enhance readability we would advice to capitalize such variables.

In [ ]:
USER = environ['JUPYTERHUB_USER']
NAME = re.sub(r'\W+', '-', USER).lower()
rubbish = 'latest'

BUILD_CONTEXT = f"buildcontext-{rubbish}"

EXPERIMENT_NAME = f'Github Issues {USER}'
DOCKER_REGISTRY = 'experiment-harbor.svc.cluster-red.antoncloud.superhub.io'
DOCKER_REGISTRY_SECRET = 'experiment-harbor-pull-secret'
# DOCKER_TAG = 'latest'
DOCKER_TAG = rubbish
TRAINING_IMAGE = f"{DOCKER_REGISTRY}/library/training:{DOCKER_TAG}"
SERVING_IMAGE = f"{DOCKER_REGISTRY}/library/seldon:{DOCKER_TAG}"

S3_ENDPOINT = 'https://experiment-minio.app.cluster-red.antoncloud.superhub.io/'
# we want to create a unique bucket for each user
S3_BUCKET = f"{NAME}-3-bucket" 
PVC_NAME = f"{S3_BUCKET}-volume"
MOUNT_PATH = f"/mnt/s3"
TRAINING_DIR = f"{MOUNT_PATH}/training-{rubbish}"

NUMPY_VER= '1.16.2'

NAMESPACE = k8s.current_namespace()

s3 = boto3.session.Session().client(
    service_name='s3',
    aws_access_key_id=get_secret('ACCESS_KEY'),
    aws_secret_access_key=get_secret('SECRET_KEY'),
    endpoint_url=S3_ENDPOINT
)


client = kfp.Client()

# try:
exp = client.get_experiment(experiment_name=EXPERIMENT_NAME)
# except:
#     exp = client.create_experiment(EXPERIMENT_NAME)

## Create bucket


Here we will generate a new bucket and create a `pvc` that will represent it as a file system inside of the wofkflow pod. To do this we need to define few variables
- `S3_BUCKET` - name of the bucket to create. By defaults we mutate name of the user
- `NAMESPACE` - points to the current namespace.
- `PVC_NAME` - derived form `S3_BUCKET`. This is a kubernetes `pvc` name. This name will be used by pipeline container (`ContainerOp` objects)

In [ ]:
from botocore.exceptions import ClientError

try:
    s3.create_bucket(Bucket=S3_BUCKET, ACL='private')
except ClientError as e:
    if e.response['Error']['Code'] != 'BucketAlreadyOwnedByYou':
        raise e

%templatefile templates/bucket-pvc.yaml -o bucket-volume.yaml
!kubectl apply -f bucket-volume.yaml

### Prepare dockerfile templates

Docker images can be rendered via `%%template` or `%templatefile` magics (source code [here](extensions/magics/templates.py)). It can intelligently use mustache `{{placeholder}}` templating syntax. Content will be replaced by the user namespace defined variable or system environment variable

You can use flags with the magic function:
* `-v` - to see content of rendered file. 
* `-h` - for more options

In [ ]:
%%template Dockerfile.keras
ARG buildfrom=python:3.6
FROM ${buildfrom}
ENV PATH "/src:${PATH}"
COPY src/keras-requirements.txt /tmp/requirements.txt

WORKDIR /tmp

RUN pip3 uninstall --yes numpy \
    && pip3 install --upgrade --no-cache-dir pip

RUN apt-get update && apt-get install -y --no-install-recommends \
    python-pandas git \
    && git clone https://github.com/google/seq2seq.git \
    && pip3 install -U -r requirements.txt \
    && rm -rf /var/lib/apt/lists/* /tmp/* /var/tmp/* \
       /usr/share/man /usr/share/doc /usr/share/doc-base

COPY src /src
WORKDIR /src

ENTRYPOINT /usr/local/bin/python

### Define pipeline to build images

Define build pipeline. Yes, we arguably using KFP to build images  that will be de-facto used by final pipeline.

We use [Kaniko](https://github.com/GoogleContainerTools/kaniko) and Kubernetes to handle build operations. Build status can be tracked via KFP pipeline dashboard

In fact build image job can be even combined with primary pipeline as physically it will be different Kubernetes pods. However for sake of general purpose efficiency we schedule build process via separate pipeline step

In [ ]:
@dsl.pipeline(
  name='Pipeline images',
  description='Build images that will be used by the pipeline'
)
def build_images(image, build_context, dockerfile):
    dsl.ContainerOp(
        name='build-image',
        image='gcr.io/kaniko-project/executor:latest',
        arguments=['--cache',
                   '--destination', image,
                   '--dockerfile', dockerfile,
                   '--context', build_context]
    ).apply(
        # docker registry credentials 
        kaniko.use_pull_secret_projection(secret_name=DOCKER_REGISTRY_SECRET)
    ).apply(
        # s3 bucket volume clame has been injected here        
        extensions.pv.use_pvc(name=PVC_NAME, mount_to=MOUNT_PATH)
    )
        
Compiler().compile(build_images, 'kaniko.tar.gz')

Compiler transforms Python DSL into an [Argo Workflow](https://argoproj.github.io/docs/argo/readme.html). And stores generated artifacts in `kaniko.tar.gz`. So it could be executed multiple times. Perhaps with different parameters

Next section will upload all files to s3, to share access with the pipeline. Files that should be ignored can be customized in [kanikoignore.txt](./kanikoignore.txt). To understand upload scenario you can review and modify: [aws.py](./extensions/kaniko/aws.py)

In [ ]:
bucket_keys = s3.list_objects(
    Bucket=S3_BUCKET,
    Prefix=BUILD_CONTEXT
).get('Contents')

aws.upload_to_s3(
    destination=f"s3://{S3_BUCKET}/{BUILD_CONTEXT}",
    ignorefile='kanikoignore.txt',
    workspace='.',
    s3_client=s3
)

run = client.run_pipeline(exp.id, 'Build docker images', 'kaniko.tar.gz', 
                          params={
                              'image': TRAINING_IMAGE,
                              'build-context': f"{MOUNT_PATH}/{BUILD_CONTEXT}",
                              'dockerfile': 'Dockerfile.keras',
                          })

Build process can be long a long term. Because often images that has been used for data science tasks are huge. In this case you might want to adjust `timeout` parameter

In [ ]:
%%time
# block until job completion
print(f"Waiting for run: {run.id}...")
result = client.wait_for_run_completion(run.id, timeout=720).run.status
print(f"Finished with: {result}")

# Data Preparation

In this chapter we will define a pipeline that will do two important steps. It will download a data set in CSV file format (we call this operation **data import**) and 

In [ ]:
IMPORT_DATASET = True

def training_op(name, arguments=[]):
    """ A template function to encapsulate similar container ops
    """
    return dsl.ContainerOp(
        name=name,
        image=TRAINING_IMAGE,
        command=['/usr/local/bin/python'],
        arguments=arguments
    ).apply(
        extensions.pv.use_pvc(name=PVC_NAME, mount_to=MOUNT_PATH)
    )
    

@dsl.pipeline(
  name='Data preparation',
  description="""Extract validate transform and load data into object storage. 
  So it could be accessible by the actual training
  """
)
def prepare_data(
    import_from: dsl.PipelineParam, 
    dataset_file: dsl.PipelineParam, 
    workdir: dsl.PipelineParam,
    model_file: dsl.PipelineParam,
    sample_size: dsl.PipelineParam=dsl.PipelineParam(name='sample-size', value='200'),
    learning_rate: dsl.PipelineParam=dsl.PipelineParam(name='learning-rate', value=0.001),
):  
    
    # Generates the training and test set. Only processes "sample-size" rows.
    process_data = training_op(
        name='process-data',
        arguments=[
            'process_data.py', 
            '--input_csv', f"{workdir}/{dataset_file}",
            '--sample_size', sample_size,
            '--output_traindf_csv', f"{workdir}/traindf.csv", 
            '--output_testdf_csv', f"{workdir}/testdf.csv"
        ]
    )
    
    # Preprocess for deep learning
    preproc_for_ml = training_op(
        name = 'preproc-for-ml',
        arguments=[
            'preproc.py',
            '--input_traindf_csv', f"{workdir}/traindf.csv",
            '--output_body_preprocessor_dpkl', f"{workdir}/body_preprocessor.dpkl",
            '--output_title_preprocessor_dpkl', f"{workdir}/title_preprocessor.dpkl",
            '--output_train_title_vecs_npy', f"{workdir}/train_title_vecs.npy",
            '--output_train_body_vecs_npy', f"{workdir}/train_body_vecs.npy",
        ]
    )
    preproc_for_ml.after(process_data)
    
    # Training
    training = training_op(
        name = 'training',
        arguments=[
            'train.py',
            '--input_body_preprocessor_dpkl', f"{workdir}/body_preprocessor.dpkl",
            '--input_title_preprocessor_dpkl', f"{workdir}/title_preprocessor.dpkl",
            '--input_train_title_vecs_npy', f"{workdir}/train_title_vecs.npy",
            '--input_train_body_vecs_npy', f"{workdir}/train_body_vecs.npy",
            '--script_name_base', f"/tmp/seq2seq",
            '--output_model_h5', f"{workdir}/{model_file}",
            '--learning_rate', learning_rate,
           '--tempfile', "True",
        ]
    )
    training.after(preproc_for_ml)
    
    # we put optional component def  on the bottom
    # so we could properly orient our DAG
    if IMPORT_DATASET:
        import_data = dsl.ContainerOp(
            name='import-data',
            image='appropriate/curl',
            arguments=['-#Lv', '--create-dirs', '-o', f"{workdir}/{dataset_file}", import_from]
        ).apply(
            pv.use_pvc(name=PVC_NAME, mount_to=MOUNT_PATH)
        )        
        process_data.after(import_data)

Compiler().compile(prepare_data, 'preproc.tar.gz')

Code below will run a pipeline and inject some pipeline parameters. Here we provide two versions of data sets
* `SAMPLE_DATA_SET` - Data set that has just over 2 megabytes. Not enough for sufficient training. However ideal for development, because of faster feedback.
* `FULL_DATA_SET` - Precreated data set with all github issues. 3 gigabytes. Good enough for sufficient model

Depending on your needs you can choose one or another data set and pass it as a pipeline parameter `data-set`

In [ ]:
# github issues small: 2Mi data set (best for dev/test)
SAMPLE_DATA_SET = 'https://s3.us-east-2.amazonaws.com/asi-kubeflow-models/gh-issues/data-sample.csv'
# data set for 3Gi. (best for training)
FULL_DATA_SET = 'https://s3.us-east-2.amazonaws.com/asi-kubeflow-models/gh-issues/data-full.csv'

run = client.run_pipeline(exp.id, 'Prepare data', 'preproc.tar.gz',
                          params={
                              'import-from': SAMPLE_DATA_SET,
                              'workdir': TRAINING_DIR,
                              'dataset-file': 'dataset.csv',
                              'model-file': 'training1.h5',
                              'learning-rate': 0.001,
                          })

In [ ]:
%%time
# block until job completion
print(f"Waiting for run: {run.id}...")
result = client.wait_for_run_completion(run.id, timeout=720).run.status
print(f"Finished with: {result}")

# Serving with Seldon
Prepping a container for serving.

In [ ]:
%%template Dockerfile.seldon
FROM seldonio/seldon-core-s2i-python3

COPY src/serving.py /microservice/IssueSummarizationModel.py
COPY src/seq2seq_utils.py /microservice
COPY src/requirements.txt /requirements.txt

RUN pip3 uninstall --yes numpy \
 && pip3 install --upgrade --no-cache-dir pip \
 && pip3 install --upgrade --no-cache-dir -r /requirements.txt

ENTRYPOINT ["python","-u","microservice.py","IssueSummarizationModel","REST","--service-type","MODEL","--persistence","0"]

## Build Serving container

To be able to serve trained model we build an image with our serving microservice. To achieve this we reuse our kaniko pipeline defined above

In [ ]:
aws.upload_to_s3(
    destination=f"s3://{S3_BUCKET}/{BUILD_CONTEXT}",
    ignorefile='kanikoignore.txt',
    workspace='.',
    s3_client=s3,
)

run = client.run_pipeline(exp.id, 'Build a serving image', 'kaniko.tar.gz', 
                          params={
                              'image': SERVING_IMAGE,
                              'build-context': f"{MOUNT_PATH}/{BUILD_CONTEXT}",
                              'dockerfile': 'Dockerfile.seldon',
                          })

In [ ]:
%%time
# block until job completion
print(f"Waiting for run: {run.id}...")
result = client.wait_for_run_completion(run.id, timeout=720).run.status
print(f"Finished with: {result}")

Then we render our `SeldonDeployment` template and deploy it with `kubectl`, similar as we have done before with `pvc` definition. Here we define reference to the model that will be used for serving

In [ ]:
MODEL_NAME = NAME
MODEL_VERSION = f"v{rubbish}"
MODEL_FILE = f"{TRAINING_DIR}/training1.h5"
TITLE_PP_FILE = f"{TRAINING_DIR}/title_preprocessor.dpkl"
BODY_PP_FILE = f"{TRAINING_DIR}/body_preprocessor.dpkl"
REPLICAS = 1

%templatefile templates/seldon.yaml -o seldon.yaml -v
# !kubectl apply -f seldon.yaml

# Tear down

Uppon completion, let's tear everything down

In [ ]:
# !kubectl delete -f bucket-volume.yaml
# !kubectl delete -f seldon.yaml